# Inversion example of diffusion + tracer mineral (uranium salt) dissolution inversion in 2d

- Importation of the required modules

In [ ]:
from pathlib import Path
import os
import copy
import tempfile
import pyrtid
import pyrtid.forward as dmfwd
import pyrtid.inverse as dminv
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LogNorm
from IPython.display import display, Image

import numpy as np

In [ ]:
pyrtid.utils.show_versions()

- Define some configurations for the plots

In [ ]:
# Some configs for the plots
new_rc_params = {
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica", "DejaVu Sans"],
    "font.size": 16,
    "text.usetex": False,
    "savefig.format": "svg",
    "svg.fonttype": "none",  # to store text as text, not as path
    "savefig.facecolor": "w",
    "savefig.edgecolor": "k",
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "figure.facecolor": "w",
}
csfont = {"fontname": "Comic Sans MS"}
hfont = {"fontname": "Helvetica"}
plt.rcParams.update(new_rc_params)

- Create a directory to store the exported figures

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
fig_save_path = Path(ipynb_path, "exported_figures")
fig_save_path.mkdir(parents=True, exist_ok=True)  # make sure that the directory exists

## Forward problem

- Define a diffusion problem with a tracer mineral dissolving

In [ ]:
# We define a function to easily generate a model.
# System parameters
nx = 57  # number of voxels along the x axis
ny = 57  # number of voxels along the y axis
dx = 5.0  # voxel dimension along the x axis [m]
dy = 5.0  # voxel dimension along the y axis [m]
nt = 4000  # number of time steps
dt = 400.0  # timestep in seconds
# Hydro parameters
D0 = 1e-5  # general initial diffusion coefficient [m2/s]
k0 = 1e-5  # general permeability
w0 = 0.23  # general porosity [fraction]
# Chemistry parameters
c0 = 0.0  # general initial concentration [molal]
M0 = 0.001  # mineral grade [mol/kg] -> kg of water
kv = -2.2e-9  # kinetic rate,       [mol/m2/s]
As = 13.5  # specific area,      [m2/mol]
Ks = 1.0 / pow(10, 3.2)  # solubility constant [no unit]
wadv = 0.0  # No avection
wmin = 1.0  # Mineral dissolution


def create_base_model() -> ForwardModel:
    return ForwardModel(
        nx, ny, dx, dy, nt, dt, c0, D0, k0, w0, M0, kv, As, Ks, wadv=wadv, wmin=wmin
    )

- Create two models

In [ ]:
model_reference = create_base_model()
model_estimate = create_base_model()

The mineral grades are defined in [mol/kg]. We calculate the conversion factor to obtain ppm and perform easier mass balances.

ConvU: parameter for converting the mineral content in [mol/kg] to metal grade in [ppm]. Note that the ConvU parameter is specific to the Uranium carrier phase: here Uraninite.

$C_{Uraninite}[\frac{mol}{kg}] = convU * T_{Uranium}[ppm] / 1000$


and


$conv_u = \frac{1.023 * density_{rock}}{238*porosity_{rock}}$

Note the 1.023 is the conversion factor from molar mass to molal mass in CHESS.

We need to interpolate to go from a grid (nx-1, ny-1) of porosity to a grid (nx, ny) of mineral grades (this is due to finite differences inplementation).

In [ ]:
porosity = np.zeros((nx, ny))
porosity[1:-1, 1:-1] = (
    model_reference.w[:-1, :-1]
    + model_reference.w[1:, :-1]
    + model_reference.w[:-1, 1:]
    + model_reference.w[1:, 1:]
) / 4
porosity[0, :] = porosity[1, :]
porosity[-1, :] = porosity[-2, :]
porosity[:, 0] = porosity[:, 1]
porosity[:, -1] = porosity[:, -2]

conv_u: np.ndarray = 1.023 * 1.63 / (238.0 * porosity)

- Create an initial gaussian spatial distribution for the mineral

In [ ]:
# Create a Gaussian Covariance Model just for the example
# To vary the results, change the seed :)
seed = 194
model_reference.s_init = create_initial_ensemble(
    n_ensemble=1,
    var=2.0,
    len_scale=10,
    nx=nx,
    ny=ny,
    min_=0.001,
    max_=0.02,
    log_scaling=False,
    seed=seed,
).reshape(nx, ny)

# Initial estimate = an homogenous value
model_estimate.s_init = np.ones((nx, ny)) * 0.005

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 6)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)
plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True": model_reference.s_init / conv_u * 1000,
        "Estimated": model_estimate.s_init / conv_u * 1000,
    },
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet")},
    xlabel="Node x #",
    ylabel="Node y #",
)
plotter.subfigs["fig0"].suptitle("Metal grade [ppm]", fontweight="bold")

fname = "Diffusion_field_true_vs_estimated"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

In [ ]:
# Conversion
convwater = 270.0 * 1000.0 * 0.997  # this take the molar mass into account
# T [mg/L] = T [mol/kg] * Mw_U [mg/mol] * rho_w [kg/L]
# Mw_U = 270*1e3 mg/mol - masse molaire de T
# rho_w = 997 kg/m3  = 0.997 kg/L - densité de l’eau

We can then run these too models and compare the results

In [ ]:
solver_true = ForwardModel(model_reference)
solver_true.solve()

solver_estimate = ForwardModel(model_estimate)
solver_estimate.solve()

We can disply the results in different ways.

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharey=True, sharex=True)
    },
)
plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2", "ax1-3"],
    data={
        "Initial": model_reference.c_init,
        "Final - True": model_reference.tr_model.conc[:, :, -1],
        "Final - Estimated": model_estimate.tr_model.conc[:, :, -1],
    },
    cbar_title="Concentration $[molal]$",
    xlabel="Node x #",
    ylabel="Node y #",
)
plotter.subfigs["fig0"].suptitle("Tracer concentration", fontweight="bold")
plotter.fig

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (11, 5)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)
nb_frames = 40
plotter.plot_2d_animated_field(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True concentration": model_reference.tr_model.conc,
        "Estimated concentration": model_estimate.tr_model.conc,
    },
    cbar_title="Concentration $[molal]$",
    xlabel="Node x #",
    ylabel="Node y #",
    nb_frames=nb_frames,
)
plotter.subfigs["fig0"].suptitle("Tracer concentration", fontweight="bold")

plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname = fig_save_path.joinpath("conc_2d_annimation.gif")
plotter.animation.save(fname, writer="pillow", fps=5)

# Display the animaiton
with open(fname, "rb") as file:
    width, height = plotter.fig.get_size_inches() * plotter.fig.dpi
    display(
        Image(file.read(), format="png", unconfined=False, width=width, height=height)
    )

- Creation of a network of injectors and producers in shape of hexagons

In [ ]:
# locations in the grid
# selection = index of cells to select. Cells are sorted by y, and if that's equal by x (increasing order).
inj_grid_locations, prod_grid_locations, polygons = gen_wells_coordinates(
    -25.0, 65.0, 175.0, 235.0, radius=40, rotation=-30, selection=[0, 1, 2, 4, 5, 6, 7]
)

In [ ]:
# Create a Gaussian Covariance Model just for the example
# To vary the results, change the seed :)
plotter = FieldPlotter(fig_params={"figsize": (8, 7)})
plotter.plot_2d_field(
    ax_names=["ax1-1"],
    data={"True": model_reference.s_init / conv_u * 1000},
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={
        "cmap": plt.get_cmap("jet"),
        "extent": [0.0, nx * dx, 0.0, ny * dy],
        "alpha": 0.5,
    },
    xlabel="Position x [m]",
    ylabel="Position y [m]",
)
plotter.subfigs["fig1-1"].suptitle("Metal grade [ppm]", fontweight="bold")

for i, j in inj_grid_locations:
    plotter.ax_dict["ax1-1"].plot(i, j, "ko")

for i, j in prod_grid_locations:
    plotter.ax_dict["ax1-1"].plot(i, j, "ro")

patches = []
for polygon in polygons:
    # Add the polygon to the collection of patches
    xy = np.array(polygon)
    patches.append(Polygon(xy, closed=True, facecolor=None))
    # Plot the number of the polygon
p = PatchCollection(patches, alpha=0.5)
p.set_facecolors("white")
p.set_edgecolors("darkgrey")
p.set_linewidth(0.2)
p.set_linestyle("-")
plotter.ax_dict["ax1-1"].add_collection(p)

for i, prod_coords in enumerate(prod_grid_locations):
    plotter.ax_dict["ax1-1"].text(*prod_coords, i)


fname = "Diffusion_field_true_vs_estimated"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

plotter.fig

In [ ]:
# Indicate the order in which to plot
obs_plot_locations = ["ax2-3", "ax1-2", "ax3-2", "ax2-2", "ax1-1", "ax3-1", "ax2-1"]

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (10, 8)},
    subfigs_params={
        "ncols": 3  # np.unique(np.array(prod_grid_locations)[:, 0].round()).size,
    },
    subplots_mosaic_params={
        "left_col": dict(
            mosaic=[["ghost1"], ["ax1-1"], ["ax1-2"], ["ghost2"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
        "centered_col": dict(
            mosaic=[["ax2-1"], ["ax2-2"], ["ax2-3"]], sharey=True, sharex=True
        ),
        "right_col": dict(
            mosaic=[["ghost3"], ["ax3-1"], ["ax3-2"], ["ghost4"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
    },
)

# hide axes and borders
plotter.ax_dict["ghost1"].axis("off")
plotter.ax_dict["ghost2"].axis("off")
plotter.ax_dict["ghost3"].axis("off")
plotter.ax_dict["ghost4"].axis("off")

# Static plot
for (x, y), ax_name in zip(prod_grid_locations, obs_plot_locations):
    ix = int(x // model_reference.dx)
    jx = int(y // model_reference.dy)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ \n x={x:.1f}m, y={y:.1f}m"

    plotter.plot_1d_field(
        ax_name=ax_name,
        title=obs_well_name,
        data={
            "True": {
                "data": model_reference.tr_model.conc[ix, jx, :],
                "kwargs": {"c": "b"},
            },
            "Initial estimation": {
                "data": model_estimate.tr_model.conc[ix, jx, :],
                "kwargs": {"c": "r"},
            },
        },
        xlabel="Time",
    )

plotter.add_fig_legend(ncol=2)
plotter.fig

## Inversion

### Gradient verification with finite difference

Inversion with three wells. Let's first check if the gradient with ths adjoint state method is correct.

In [ ]:
# Create an executor
executor = InversionExecutor(copy.copy(model_estimate))

param = AdjustableParameter(
    name=ParameterName.INITIAL_GRADE,
    lbounds=1e-3,
    ubounds=2e-2,
)

observables = {}
# Static plot
for x, y in prod_grid_locations:
    ix = int(x // model_reference.dx)
    jx = int(y // model_reference.dy)

    vals = model_reference.tr_model.conc[ix, jx, :]
    timesteps = np.arange(vals.shape[0])

    observables[(ix, jx)] = Observable(
        state_variable="tracer",
        location=(slice(ix, ix + 1, 1), slice(jx, jx + 1, 1)),
        timesteps=timesteps,
        values=vals,
    )

is_grad_ok = executor.is_gradient_correct(
    param,
    list(observables.values()),
)
print("Is the gradient correct: ", is_grad_ok)

The gradient seems correct, let's plot it !

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (16, 5)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharey=True, sharex=True)
    },
)

adjoint_grad = executor.rt_solver.get_gridded_gradients(
    ParameterName.INITIAL_GRADE, is_adjoint=True
)[:, :, 0]
# fd_grad = executor.rt_solver.get_gridded_gradients(ParameterName.DIFFUSION, is_adjoint=True)[:, :, 0]
fd_grad = executor.rt_solver.inverse_model.list_g_fd_res[0].reshape(
    executor.rt_solver.model.nx - 1, -1
)

plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2", "ax1-3"],
    data={
        "Adjoint": adjoint_grad,
        "Finite differences": fd_grad,
        "Residuals": adjoint_grad - fd_grad,
    },
    cbar_title="Diffusion gradient $[molal]$",
    is_symmetric_cbar=True,
)
plotter.subfigs["fig0"].suptitle("Gradient at first iteration", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("Node x #", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Node y #", fontweight="bold")
plotter.fig

Plot of the adjoint variables

In [ ]:
plotter = FieldPlotter()
plotter.plot_2d_field(
    ax_names=["ax1-1"],
    data={
        "Adjoint variable": executor.rt_solver.inverse_model.adj_conc[
            :, :, -500
        ],  # adjoint variables are reversed in time
        # "Adjoint variable": np.flip(executor.rt_solver.inverse_model.adj_conc, axis=2), # adjoint variables are reversed in time
        # "Adjoint variable": executor.rt_solver.inverse_model.adj_conc[0, :, :], # adjoint variables are reversed in time
    },
    cbar_title="Adj conc $[molal]$",
    # Plot with a log scale
    # imshow_kwargs={"norm": LogNorm(vmin=executor.rt_solver.inverse_model.adj_conc.min(), vmax=executor.rt_solver.inverse_model.adj_conc.max())}
)
plotter.subfigs["fig1-1"].suptitle("Adj tracer concentration", fontweight="bold")
plotter.subfigs["fig1-1"].supxlabel("Node x #", fontweight="bold")
plotter.subfigs["fig1-1"].supylabel("Node y #", fontweight="bold")
plotter.fig

In [ ]:
plotter = FieldPlotter()
nb_frames = 30
import matplotlib as mpl

# Set bad (see https://stackoverflow.com/questions/9455044/problems-with-zeros-in-matplotlib-colors-lognorm)
# my_cmap = copy.copy(mpl.cm.get_cmap("bwr")) # copy the default cmap
# my_cmap.set_bad((0, 0, 0))

data = (
    executor.rt_solver.inverse_model.adj_conc[:, :, ::-1] + 1e-1
)  # to avoid zeros with a logscale
plotter.plot_2d_animated_field(
    ax_names=["ax1-1"],
    data={
        "Adjoint variable": data,  # adjoint variables are reversed in time
        # "Adjoint variable": np.flip(executor.rt_solver.inverse_model.adj_conc, axis=2), # adjoint variables are reversed in time
        # "Adjoint variable": executor.rt_solver.inverse_model.adj_conc[0, :, :], # adjoint variables are reversed in time
    },
    cbar_title="Adj conc $[molal]$",
    nb_frames=nb_frames,
    # Plot with a log scale
    imshow_kwargs={"norm": LogNorm(vmax=np.max(data))},
)
plotter.subfigs["fig1-1"].suptitle("Adj tracer concentration", fontweight="bold")
plotter.subfigs["fig1-1"].supxlabel("Node x #", fontweight="bold")
plotter.subfigs["fig1-1"].supylabel("Node y #", fontweight="bold")

plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname = fig_save_path.joinpath("adj_conc_2d_annimation.gif")
plotter.animation.save(fname, writer="pillow", fps=2)

# Display the animaiton
with open(fname, "rb") as file:
    width, height = plotter.fig.get_size_inches() * plotter.fig.dpi
    display(
        Image(file.read(), format="png", unconfined=False, width=width, height=height)
    )

At the observatinon well, we can see that the sign of the adjoint variable globally depends on the sign of the residuals.

In [ ]:
plotter = StateVariableSlicePlotter(nb_obs_loc=4, nb_obs_type=2)

for i, (coords, obs) in enumerate(observables.items()):
    ix = coords[0]
    jx = coords[1]
    x = (ix + 0.5) * model_reference.dx
    y = (jx + 0.5) * model_reference.dy
    obs_well_name = f"obs. well @ node (x={x}m) (y={y}m)"

    plotter.plot_state(
        row_id=i,
        obs_well_name=obs_well_name,
        data={
            "Tracer": {
                "True obs.": (
                    executor.rt_solver.inverse_model.observables[i].values,
                    {"c": "b"},
                ),
                "Initial estimation": (
                    executor.rt_solver.model.fwd_conc[ix, jx, :],
                    {"c": "r"},
                ),
            },
            "Tracer adjoint variable": {
                "Adjoint var.": (
                    executor.rt_solver.inverse_model.adj_conc[ix, jx, :],
                    {"c": "g"},
                ),
            },
        },
    )
plotter.add_fig_legend(ncol=2)

plotter.fig

### First inversion run

In [ ]:
# Create an executor
executor = InversionExecutor(copy.copy(model_estimate))

param = AdjustableParameter(
    name=ParameterName.INITIAL_GRADE,
    lbounds=1e-3,
    ubounds=2e-2,
)

observables = {}
# Static plot
for x, y in prod_grid_locations:
    ix = int(x // model_reference.dx)
    jx = int(y // model_reference.dy)
    x = (ix + 0.5) * model_reference.dx
    y = (jx + 0.5) * model_reference.dy
    obs_well_name = f"obs. well @ node (x={x}m) (y={y}m)"

    vals = model_reference.tr_model.conc[ix, jx, :]
    timesteps = np.arange(vals.shape[0])

    observables[(ix, jx)] = Observable(
        state_variable="tracer",
        location=(slice(ix, ix + 1, 1), slice(jx, jx + 1, 1)),
        timesteps=timesteps,
        values=model_reference.tr_model.conc[ix, jx, :],
    )

executor.run_inversion(
    param,
    list(observables.values()),
    solver_name="L-BFGS-B",
    solver_options={"maxfun": 30, "maxiter": 30, "ftol": 1e-5, "gtol": 1e-5},
    is_check_gradient=False,
)

Let's plot the results: evolution of the parameter, the gradient, the objective function.

In [ ]:
# Here comes the python code
nx = executor.rt_solver.model.nx
ny = executor.rt_solver.model.ny
adjoint_gradients = executor.rt_solver.get_gridded_gradients(
    ParameterName.INITIAL_GRADE, is_adjoint=True
)
nb_frames = adjoint_gradients.shape[-1]

anim_res_plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (13, 8)},
    subfigs_params={"nrows": 2, "ncols": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharex=True, sharey=True),
        "fig1": dict(mosaic=[["ax2-1", "ax2-2"]]),
    },
)

# 1 frame per solver iteration
nb_frames: int = len(executor.rt_solver.inverse_model.loss_scaled_history)

# 1) Gradient evolution
anim_res_plotter.plot_2d_animated_field(
    ax_names=["ax1-1"],
    data={
        "Gradient": adjoint_gradients,
    },
    is_add_grid=True,
    # title="Gradient",
    xlabel="Node x#",
    ylabel="Node y#",
    nb_frames=nb_frames,
    is_symmetric_cbar=True,
)

# 2) Parameter evolution
anim_res_plotter.plot_2d_animated_field(
    ax_names=["ax1-2", "ax1-3"],
    data={
        "True": np.repeat(
            model_reference.s_init.reshape(nx, ny, -1), nb_frames, axis=2
        ),
        "inverted": np.hstack(param.archived_values).reshape(-1, nx, ny).T,
    },
    # title="Gradient",
    xlabel="Node x#",
    ylabel="Node y#",
    nb_frames=nb_frames,
    cbar_title="Diffusion coefficient $[m/s]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet")},
)

# 3) Objective function
vals = executor.rt_solver.inverse_model.loss_scaled_history
obj_fun_vals = np.full((nb_frames, len(vals)), fill_value=np.nan)
for i in range(len(vals)):
    obj_fun_vals[i, : i + 1] = vals[: i + 1]

anim_res_plotter.plot_1d_animated_field(
    ax_name="ax2-1",
    nb_frames=nb_frames,
    data={
        "Obj fun": {"data": obj_fun_vals.T, "kwargs": {"c": "r", "linestyle": "--"}},
    },
    title="Objective function",
    xlabel="Iteration #",
)
anim_res_plotter.ax_dict["ax2-1"].set_yscale("log")
# grad_plotter.add_fig_legend(ncol=2)
# grad_plotter.fig

# 4) Observation vs predicted values
anim_res_plotter.plot_observed_vs_simulated(
    ax_name="ax2-2",
    obs_vector=executor.rt_solver.inverse_model.get_obs_values_as_1d_vector() * 1000,
    pred_vector=executor.rt_solver.get_results_matching_obs_vector() * 1000,
    # pred_vector_initial=model_estimate.get_results_matching_obs_vector() * 1000,
    unit="$mmol.l^{-1}$",
)

anim_res_plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname = fig_save_path.joinpath("m_j_g_animation.gif")
anim_res_plotter.animation.save(fname, writer="pillow", fps=2)

# Display the animaiton
with open(fname, "rb") as file:
    width, height = anim_res_plotter.fig.get_size_inches() * anim_res_plotter.fig.dpi
    display(
        Image(file.read(), format="png", unconfined=False, width=width, height=height)
    )

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (10, 10)},
    subfigs_params={
        "nrows": max([elt[0] for elt in obs_plot_locations]),
        "ncols": max([elt[1] for elt in obs_plot_locations]),
    },
)

# Static plot
for (ix, jx), (row_id, col_id) in zip(obs_grid_locations, obs_plot_locations):
    x = (ix + 0.5) * model_reference.dx
    y = (jx + 0.5) * model_reference.dy
    obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x}m) (y={y}m)"

    plotter.plot_1d_field(
        ax_name=f"ax{row_id}-{col_id}",
        title=obs_well_name,
        data={
            "True": {
                "data": model_reference.tr_model.conc[ix, jx, :],
                "kwargs": {"c": "g"},
            },
            "Initial estimation": {
                "data": model_estimate.tr_model.conc[ix, jx, :],
                "kwargs": {"c": "r"},
            },
            "Inverted": {
                "data": executor.rt_solver.model.fwd_conc[ix, jx, :],
                "kwargs": {"c": "b"},
            },
        },
        xlabel="Time",
    )
plotter.add_fig_legend(ncol=2)
plotter.fig

# Impact of the regularization

We create three different models:
- The first without regularization
- The second with a tickonov regularization
- The third with a TV regularization

In [ ]:
from pyrtid.utils import RegularizationType

param_no_reg = AdjustableParameter(
    name="diffusion",
    lbounds=2e-5,
    ubounds=5e-3,
    preconditioner=dminv.LogTransform()
)

param_reg_tikhonov = AdjustableParameter(
    name="diffusion",
    lbounds=2e-5,
    ubounds=5e-3,
    preconditioner=dminv.LogTransform()
    regularization=RegularizationType.TIKHONOV,
)

param_reg_tv = AdjustableParameter(
    name="diffusion",
    lbounds=2e-5,
    ubounds=5e-3,
    preconditioner=dminv.LogTransform()
    regularization=RegularizationType.TOTAL_VARIATION,
)

In [ ]:
# Common arguments for solvers
common_kwargs = dict(
    solver_name="L-BFGS-B",
    solver_options={"maxfun": 20, "maxiter": 20, "ftol": 1e-4, "gtol": 1e-4},
    is_check_gradient=False,
)

# Running the inversion without regularization
executor_no_reg = InversionExecutor(model_estimate)
executor_no_reg.run_inversion(param_no_reg, list(observables.values()), **common_kwargs)

# Running the inversion with tikhonov regularization
executor_tikhonov = InversionExecutor(model_estimate)
executor_tikhonov.run_inversion(
    param_reg_tikhonov, list(observables.values()), **common_kwargs
)

# Running the inversion with total variation regularization
executor_tv = InversionExecutor(model_estimate)
executor_tv.run_inversion(param_reg_tv, list(observables.values()), **common_kwargs)

Let's see the results in term of inverted parameter.

In [ ]:
# Here comes the python code
grad_plotter = FieldPlotter(nrows=1, ncols=3, sharex=True, sharey=True)

# Get the parameter to plot
# TODO: think aboutusing a dict ???
param = executor.rt_solver.inverse_model.parameters_to_adjust[0]

grad_index = -1

grad_plotter.plot_1d_field(
    ax_name="ax1-1",
    data={
        "True": {"data": model_reference.diffusion, "kwargs": {"c": "b"}},
        "Initial": {"data": param_no_reg.archived_values[0], "kwargs": {"c": "orange"}},
        "Final": {
            "data": param_no_reg.archived_values[grad_index],
            "kwargs": {"c": "r", "linestyle": "--"},
        },
    },
    title="No regularization",
    ylabel="Diffusivity (m/s)",
    xlabel="Nodes #",
)
grad_plotter.plot_1d_field(
    ax_name="ax1-2",
    data={
        "True": {"data": model_reference.diffusion, "kwargs": {"c": "b"}},
        "Initial": {
            "data": param_reg_tikhonov.archived_values[0],
            "kwargs": {"c": "orange"},
        },
        "Final": {
            "data": param_reg_tikhonov.archived_values[grad_index],
            "kwargs": {"c": "r", "linestyle": "--"},
        },
    },
    title="Tikhonov regularization",
    xlabel="Nodes #",
)
grad_plotter.plot_1d_field(
    ax_name="ax1-3",
    data={
        "True": {"data": model_reference.diffusion, "kwargs": {"c": "b"}},
        "Initial": {"data": param_reg_tv.archived_values[0], "kwargs": {"c": "orange"}},
        "Final": {
            "data": param_reg_tv.archived_values[grad_index],
            "kwargs": {"c": "r", "linestyle": "--"},
        },
    },
    title="Total variation regularization",
    xlabel="Nodes #",
)

grad_plotter.ax_dict["ax1-1"].set_yscale("log")
grad_plotter.add_fig_legend(ncol=3)
grad_plotter.fig.suptitle(
    "Impact of the regularization on the final inverted parameter"
)
grad_plotter.fig